In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop,Adam
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from tensorflow.python.keras.models import Sequential
import tensorflow as tf
from tensorflow.python.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,Activation

In [2]:
#set dir path where my images are saved
datadir="C:/Users/Asus/MENS_WEAR/train"
categories=["Jeans","Trouser"]

IMG_SIZE=200

In [3]:
#creating our training data for model training and also normalize the data for further processing.
training_data=[]

def create_training_data():
    for category in categories:
        path=os.path.join(datadir,category)
        class_num=categories.index(category)
        for img in os.listdir(path):
            try:
                
                img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()
 
    

In [4]:
#set dir path where my images are saved
data_dir="C:/Users/Asus/MENS_WEAR/validation"
categories=["Jeans","Trouser"]



In [5]:
#creating our training data for model training and also normalize the data for further processing.
validation_data=[]

def create_validation_data():
    for category in categories:
        path=os.path.join(data_dir,category)
        class_num=categories.index(category)
        for img in os.listdir(path):
            try:
                
                img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                validation_data.append([new_array,class_num])
            except Exception as e:
                pass
create_validation_data()
 
    

In [6]:
#from preventing the overfitting and underfitting we have to shuffle the data 
import random
random.shuffle(training_data)

In [7]:
x_train=[]
y_train=[]

In [8]:
#making x and y for training the model
for features,label in training_data:
    x_train.append(features)
    y_train.append(label)

In [9]:
x_validation=[]
y_validation=[]

In [10]:
#making x and y for training the model
for features,label in validation_data:
    x_validation.append(features)
    y_validation.append(label)

In [11]:
x_train=np.array(x_train)
x_validation=np.array(x_validation)

In [12]:
x_train=x_train.reshape(x_train.shape[0],IMG_SIZE,IMG_SIZE,1)
x_validation=x_validation.reshape(x_validation.shape[0],IMG_SIZE,IMG_SIZE,1)

In [13]:
x_train=x_train.astype('float32')
x_validation=x_validation.astype('float32')

In [14]:
x_train=x_train/255
x_validation=x_validation/255

In [15]:
num_class=2
y_train=tf.keras.utils.to_categorical(y_train,num_class)
y_validation=tf.keras.utils.to_categorical(y_validation,num_class)

In [16]:

batch_size=10
epochs=15
input_shape=(IMG_SIZE,IMG_SIZE,1)

In [17]:
#creating model
model=Sequential()
model.add(Conv2D(32,kernel_size=(5,5),strides=(1,1),
                activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(64,kernel_size=(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(num_class,activation='sigmoid'))

model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 196, 196, 32)      832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 94, 94, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 141376)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                9048128   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1

In [18]:
from tensorflow.keras.optimizers import Adam,RMSprop,Nadam,SGD
#from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau


In [19]:
def adam():
    checkpoint=ModelCheckpoint('C:/Users/Asus/MENS_WEAR/mens_wear_adam_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
adam()

Epoch 1/15
96/96 [==============================] - ETA: 0s - loss: 0.7038 - accuracy: 0.5865
Epoch 00001: val_loss improved from inf to 0.60614, saving model to C:/Users/Asus/MENS_WEAR/mens_wear_adam_cnn.h5
96/96 [==============================] - 62s 641ms/step - loss: 0.7038 - accuracy: 0.5865 - val_loss: 0.6061 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
96/96 [==============================] - ETA: 0s - loss: 0.5095 - accuracy: 0.7615
Epoch 00002: val_loss improved from 0.60614 to 0.58394, saving model to C:/Users/Asus/MENS_WEAR/mens_wear_adam_cnn.h5
96/96 [==============================] - 73s 764ms/step - loss: 0.5095 - accuracy: 0.7615 - val_loss: 0.5839 - val_accuracy: 0.7250 - lr: 0.0010
Epoch 3/15
96/96 [==============================] - ETA: 0s - loss: 0.3249 - accuracy: 0.8656
Epoch 00003: val_loss improved from 0.58394 to 0.57862, saving model to C:/Users/Asus/MENS_WEAR/mens_wear_adam_cnn.h5
96/96 [==============================] - 81s 844ms/step - loss: 0.3249 - accur

In [20]:
def Rmsprop():
    checkpoint=ModelCheckpoint('C:/Users/Asus/MENS_WEAR/mens_wear_RMSprop_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.RMSprop(),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
Rmsprop()    

Epoch 1/15
96/96 [==============================] - ETA: 0s - loss: 0.2141 - accuracy: 0.9083
Epoch 00001: val_loss improved from inf to 0.91086, saving model to C:/Users/Asus/MENS_WEAR/mens_wear_RMSprop_cnn.h5
96/96 [==============================] - 119s 1s/step - loss: 0.2141 - accuracy: 0.9083 - val_loss: 0.9109 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 2/15
96/96 [==============================] - ETA: 0s - loss: 0.1325 - accuracy: 0.9479
Epoch 00002: val_loss did not improve from 0.91086
96/96 [==============================] - 117s 1s/step - loss: 0.1325 - accuracy: 0.9479 - val_loss: 1.1053 - val_accuracy: 0.7000 - lr: 0.0010
Epoch 3/15
96/96 [==============================] - ETA: 0s - loss: 0.1200 - accuracy: 0.9583
Epoch 00003: val_loss did not improve from 0.91086
96/96 [==============================] - 104s 1s/step - loss: 0.1200 - accuracy: 0.9583 - val_loss: 1.2866 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 4/15
96/96 [==============================] - ETA: 0s - loss: 0.

In [21]:
def Nadam():
    checkpoint=ModelCheckpoint('C:/Users/Asus/MENS_WEAR/mens_wear_Nadam_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Nadam(),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
Nadam()    

Epoch 1/15
96/96 [==============================] - ETA: 0s - loss: 0.1419 - accuracy: 0.9438
Epoch 00001: val_loss improved from inf to 1.07370, saving model to C:/Users/Asus/MENS_WEAR/mens_wear_Nadam_cnn.h5
96/96 [==============================] - 114s 1s/step - loss: 0.1419 - accuracy: 0.9438 - val_loss: 1.0737 - val_accuracy: 0.8250 - lr: 0.0010
Epoch 2/15
96/96 [==============================] - ETA: 0s - loss: 0.0724 - accuracy: 0.9688
Epoch 00002: val_loss did not improve from 1.07370
96/96 [==============================] - 122s 1s/step - loss: 0.0724 - accuracy: 0.9688 - val_loss: 1.3978 - val_accuracy: 0.7000 - lr: 0.0010
Epoch 3/15
96/96 [==============================] - ETA: 0s - loss: 0.0666 - accuracy: 0.9792
Epoch 00003: val_loss did not improve from 1.07370
96/96 [==============================] - 113s 1s/step - loss: 0.0666 - accuracy: 0.9792 - val_loss: 1.2557 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
96/96 [==============================] - ETA: 0s - loss: 0.11

In [22]:
def SGD_momentum():
    checkpoint=ModelCheckpoint('C:/Users/Asus/MENS_WEAR/mens_wear_SGD_momentum_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.SGD(lr=0.001,momentum=0.9),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
SGD_momentum()    

Epoch 1/15
96/96 [==============================] - ETA: 0s - loss: 0.0902 - accuracy: 0.9698
Epoch 00001: val_loss improved from inf to 1.00827, saving model to C:/Users/Asus/MENS_WEAR/mens_wear_SGD_momentum_cnn.h5
96/96 [==============================] - 131s 1s/step - loss: 0.0902 - accuracy: 0.9698 - val_loss: 1.0083 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 2/15
96/96 [==============================] - ETA: 0s - loss: 0.0311 - accuracy: 0.9917
Epoch 00002: val_loss did not improve from 1.00827
96/96 [==============================] - 107s 1s/step - loss: 0.0311 - accuracy: 0.9917 - val_loss: 1.0705 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 3/15
96/96 [==============================] - ETA: 0s - loss: 0.0249 - accuracy: 0.9937
Epoch 00003: val_loss did not improve from 1.00827
96/96 [==============================] - 99s 1s/step - loss: 0.0249 - accuracy: 0.9937 - val_loss: 1.2161 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 4/15
96/96 [==============================] - ETA: 0s - loss

In [23]:
def NAG():
    checkpoint=ModelCheckpoint('C:/Users/Asus/MENS_WEAR/mens_wear_NAG_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.SGD(lr=0.01,nesterov=0.9),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
NAG()    

Epoch 1/15
96/96 [==============================] - ETA: 0s - loss: 0.0340 - accuracy: 0.9875
Epoch 00001: val_loss improved from inf to 1.05401, saving model to C:/Users/Asus/MENS_WEAR/mens_wear_NAG_cnn.h5
96/96 [==============================] - 106s 1s/step - loss: 0.0340 - accuracy: 0.9875 - val_loss: 1.0540 - val_accuracy: 0.8000 - lr: 0.0100
Epoch 2/15
96/96 [==============================] - ETA: 0s - loss: 0.0253 - accuracy: 0.9948
Epoch 00002: val_loss did not improve from 1.05401
96/96 [==============================] - 103s 1s/step - loss: 0.0253 - accuracy: 0.9948 - val_loss: 1.1832 - val_accuracy: 0.8000 - lr: 0.0100
Epoch 3/15
96/96 [==============================] - ETA: 0s - loss: 0.0211 - accuracy: 0.9937
Epoch 00003: val_loss did not improve from 1.05401
96/96 [==============================] - 109s 1s/step - loss: 0.0211 - accuracy: 0.9937 - val_loss: 1.3684 - val_accuracy: 0.7500 - lr: 0.0100
Epoch 4/15
96/96 [==============================] - ETA: 0s - loss: 0.0210

In [24]:
#Loading the model

import tensorflow as tf 
classifier = tf.keras.models.load_model('C:/Users/Asus/MENS_WEAR/mens_wear_NAG_cnn.h5')

In [54]:
import warnings
warnings.filterwarnings('ignore')
import cv2
import numpy as np

def draw_test(name,res,input_im):
    BLACK=[0,0,0]
    res=int(res)
    if res==0:
        pred="Jeans"
    if res==1:
        pred="Trouser"
    expanded_image=cv2.copyMakeBorder(input_im,0,0,0,imageL.shape[0]*3, cv2.BORDER_CONSTANT,value=BLACK)
    expanded_image=cv2.cvtColor(expanded_image,cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image,str(pred),(100,100),cv2.FONT_HERSHEY_COMPLEX_SMALL,4,(1,60,0),5)
    cv2.imshow(name,expanded_image)
    
    
for i in range (0,10):
    rand=np.random.randint(0,len(x_train))
    input_im=x_train[rand]
    
    imageL=cv2.resize(input_im,None,fx=4,fy=4,interpolation=cv2.INTER_CUBIC)
    input_im=input_im.reshape(1,200,200,1)
    
    #get prediction
    res=str(classifier.predict_classes(input_im,1,verbose=0)[0])
    
    draw_test("Prediction",res,imageL)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()